<a href="https://colab.research.google.com/github/gptix/data-visualization/blob/master/Coal_Plant_Bubble_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualization of intensity of use of coal to generate power in the United States in 2018.

## Data Sources

U.S. Energy Information Administration

https://www.eia.gov/



**Form EIA-923**

"The survey Form EIA-923 collects detailed electric power data -- monthly and annually -- on electricity generation, fuel consumption, fossil fuel stocks, and receipts at the power plant and prime mover level."

https://www.eia.gov/electricity/data/eia923/



**Form EIA-860**

"The survey Form EIA-860 collects generator-level specific information about existing and planned generators and associated environmental equipment at electric power plants with 1 megawatt or greater of combined nameplate capacity."

https://www.eia.gov/electricity/data/eia860/

## Software

Data Manipulation: Pandas

Vizualization: Plotly

Notebook Creation and coding: Google Colab

In [0]:
import pandas as pd
import plotly.graph_objects as go

url_base = "https://raw.githubusercontent.com/gptix/data-visualization/master/"

# set filenames
# Detailed data, including fuel type and annual production.
detail_filename = "EIA923_Schedules_2_3_4_5_M_12_2018_Final_Revision.csv"

# Plant data, including latitude and longitude.
plants_filename = "2___Plant_Y2018.csv"

detail_df = pd.read_csv(url_base + detail_filename)
plants_df = pd.read_csv(url_base + plants_filename, skiprows=1)

# detail_df.head(1)

In [0]:
# extract needed data
# filter for rows related to use of coal as fuel.
fuel_type = 'COL'
detail_df = detail_df[detail_df.AER_Fuel_Type_Code.isin([fuel_type])]
# detail_df.shape

# Re-shape detail_df to contain only columns needed.
detail_columns = detail_df.columns
detail_columns_needed = ['Plant_Id', 'Reported_Prime_Mover', 'AER_Fuel_Type_Code', 'Total_Fuel_Consumption_MMBtu', 'Net_Generation_(Megawatthours)']
detail_columns_to_drop = list(set(detail_columns) - set(detail_columns_needed))
detail_df = detail_df.drop(columns=detail_columns_to_drop)

detail_df.columns
detail_df.head(1)

# create df that summarizes fuel use and power generation by Plant.
detail_summed_df = detail_df.groupby('Plant_Id').sum()
detail_summed_df.head(2)
detail_summed_df.Total_Fuel_Consumption_MMBtu
detail_summed_df.columns
detail_summed_nonzero_df = detail_summed_df[detail_summed_df['Net_Generation_(Megawatthours)'] > 0.1]

# detail_summed_nonzero_df.shape
# detail_summed_nonzero_df.head()

In [0]:
# check for negatives, which break plotly figure.
# detail_summed_nonzero_df[detail_summed_nonzero_df['Net_Generation_(Megawatthours)'] < 0]

In [0]:
# Re-shape detail_df to contain only columns needed.
plants_columns = plants_df.columns
plants_columns_needed = ['Plant Code', 'Plant Name', 'State', 'Latitude', 'Longitude']
plants_columns_to_drop = list(set(plants_columns) - set(plants_columns_needed))
plants_df = plants_df.drop(columns=plants_columns_to_drop)
# plants_columns
# Rename column so that join will work.
plants_df = plants_df.rename(columns={"Plant Code" : "Plant_Id"})


# plants_df.head(1)

In [0]:
# Make a df of info for map.
map_data_df = plants_df.join(detail_summed_nonzero_df, on='Plant_Id')

# Replace NaN values with 0
map_data_df['Total_Fuel_Consumption_MMBtu'].fillna(0, inplace=True)
map_data_df['Net_Generation_(Megawatthours)'].fillna(0, inplace=True)

# RENAME COLUMN
map_data_df.rename(columns={"Net_Generation_(Megawatthours)": "MWh"}, inplace=True)

In [0]:
# set colors to be used for bubbles.
marker_color = 'rgb(1,   1, 10)'

# Set scaling factor for size of bubbles.
scale = 5e4

# Set text data for each point to be displayed on hover
map_data_df['text'] = map_data_df['Plant Name'] + '<br>MWh ' + map_data_df['MWh'].astype(str)

In [17]:
# Instantiate figure.
fig = go.Figure()


# Add data for bubbles to figure object
fig.add_trace(go.Scattergeo(
        
    # control type of figure
    locationmode = 'USA-states',

    # lat, long used to control placement of bubble
    lon = map_data_df['Longitude'],
    lat = map_data_df['Latitude'],
        
    # metadata to be used for each bubble (when hovering)
    text = map_data_df['text'],

    marker = dict(
        size = (map_data_df['MWh'] / scale),
        color = marker_color,
        line_color='rgb(40,40,40)',
        line_width=0.5,
        sizemode = 'area'
        ),
        name = 'Coal Burning Plants'
    ))

# Configure overall map
fig.update_layout(
        title_text = 'Coal Plants Spewing Pollution in U.S. - 2018 - Scaled by MWh',
        showlegend = True,
        geo = dict(
            scope = 'usa',
            landcolor = 'lightgreen',
        )
    )

fig.show()

![Chart Screenshot](https://raw.githubusercontent.com/gptix/data-visualization/master/Coal_Plants_US_2018.png)